In [1]:
import pandas as pd
pd.options.display.max_columns = None # Muestra todas las columnas de los dataframes
import json
import warnings
warnings.filterwarnings("ignore") # No muestra advertencias 
import re
import ast
import pyarrow




In [2]:
# Leo el siguiente dataframe: user_reviews_final.csv
df_ur1 = pd.read_parquet('Tablas/user_review_final.parquet')
df_ur1.shape
df_ur1

,user_id,item_id,helpful,recommend,review,posted_year
0,76561197970982479,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
1,js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
2,doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013
3,maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014
4,Wackky,249130,7 of 8 people (88%) found this review helpful,True,This game is Marvellous.,2014
...,...,...,...,...,...,...
48221,76561198138691719,332310,No ratings yet,True,Normally I would hardly play a lego based game...,2015
48222,SKELETRONPRIMEISOP,440,No ratings yet,True,TF2 is alot of fun and its really good but the...,2014
48223,76561198141079508,304930,No ratings yet,True,Fun game with friends,2014
48224,ShadowYT100,265630,No ratings yet,True,So Fun!! :D,2015


In [3]:
# Asigno un género a la variable genero, simulando una consulta desde la API
genero = 'Action'
# Elimino las columnas que no voy a utilizar
df_ur2 =df_ur1.drop(columns=['helpful', 'recommend', 'review' ])
# Cambio el nombre de la columna item_id por id
df_ur2.rename(columns={'item_id': 'id'}, inplace=True)
df_ur2

,user_id,id,posted_year
0,76561197970982479,1250,2011
1,js41637,251610,2014
2,doctr,250320,2013
3,maplemage,211420,2014
4,Wackky,249130,2014
...,...,...,...
48221,76561198138691719,332310,2015
48222,SKELETRONPRIMEISOP,440,2014
48223,76561198141079508,304930,2014
48224,ShadowYT100,265630,2015


In [4]:
# Elimino filas donde posted_year = 0
condicion = df_ur2['posted_year'] == 0
df_ur2.drop(df_ur2[condicion].index, inplace=True)
df_ur2


,user_id,id,posted_year
0,76561197970982479,1250,2011
1,js41637,251610,2014
2,doctr,250320,2013
3,maplemage,211420,2014
4,Wackky,249130,2014
...,...,...,...
48221,76561198138691719,332310,2015
48222,SKELETRONPRIMEISOP,440,2014
48223,76561198141079508,304930,2014
48224,ShadowYT100,265630,2015


In [5]:
df_ur2['id'] = df_ur2['id'].astype(int)

In [6]:
# Leo steam_games_output
df_sgo = pd.read_parquet('Tablas/steam_games_output_limpio.parquet')
df_sgo 


,genres,title,id,developer,release_year
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018
...,...,...,...,...,...
68605,Indie,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68606,Racing,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68607,Simulation,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
68608,Casual,EXIT 2 - Directions,658870,"xropi,stev3ns",2017


In [7]:
# Me quedo con 'genres' e 'id'
df_sgo1 =df_sgo.drop(columns=['title', 'developer', 'release_year' ])
df_sgo1


,genres,id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
68605,Indie,610660
68606,Racing,610660
68607,Simulation,610660
68608,Casual,658870


In [8]:
df_sgo1['id'] = df_sgo1['id'].astype(int)

In [9]:
# Ahora hago la unión con df_ur2, así me quedan los usuarios relacionados con el año
df_join2 = pd.merge(df_sgo1, df_ur2, on='id')
df_join2

,genres,id,user_id,posted_year
0,Action,70,EizanAratoFujimaki,2015
1,Action,70,76561198020928326,2014
2,Action,70,pandasmokingbamboo,2015
3,Action,70,matrix12384,2014
4,Action,70,coolman1342,2011
...,...,...,...,...
98412,Action,80,76561198023508728,2014
98413,Action,80,Lone_walker,2013
98414,Action,80,virex4,2011
98415,Action,80,KILLERamateur,2014


In [10]:
df_join2

,genres,id,user_id,posted_year
0,Action,70,EizanAratoFujimaki,2015
1,Action,70,76561198020928326,2014
2,Action,70,pandasmokingbamboo,2015
3,Action,70,matrix12384,2014
4,Action,70,coolman1342,2011
...,...,...,...,...
98412,Action,80,76561198023508728,2014
98413,Action,80,Lone_walker,2013
98414,Action,80,virex4,2011
98415,Action,80,KILLERamateur,2014


In [11]:
# Leo el dataframe user_items
df_ui = pd.read_parquet('Tablas/user_items_limpio.parquet')
df_ui


,user_id,item_id,item_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,30,Day of Defeat,7
2,76561197970982479,300,Day of Defeat: Source,4733
3,76561197970982479,240,Counter-Strike: Source,1853
4,76561197970982479,3830,Psychonauts,333
...,...,...,...,...
3246347,76561198329548331,304930,Unturned,677
3246348,76561198329548331,227940,Heroes & Generals,43
3246349,76561198329548331,388490,One Way To Die: Steam Edition,3
3246350,76561198329548331,521570,You Have 10 Seconds 2,4


In [12]:
# No necesito item_name. Elimino la columna
df_ui1 =df_ui.drop(columns=['item_name' ])
# cambio el nombre de la columna item_id por id
df_ui1.rename(columns={'item_id': 'id'}, inplace=True) 



In [13]:
df_ui1

,user_id,id,playtime_forever
0,76561197970982479,10,6
1,76561197970982479,30,7
2,76561197970982479,300,4733
3,76561197970982479,240,1853
4,76561197970982479,3830,333
...,...,...,...
3246347,76561198329548331,304930,677
3246348,76561198329548331,227940,43
3246349,76561198329548331,388490,3
3246350,76561198329548331,521570,4


In [14]:
df_ui1['id'] = df_ui1['id'].astype(int)

In [15]:
df_join1 = df_ui1.merge(df_sgo1, on="id", how="inner")
df_join1



,user_id,id,playtime_forever,genres
0,76561197970982479,10,6,Action
1,doctr,10,93,Action
2,corrupted_soul,10,108,Action
3,WeiEDKrSat,10,328,Action
4,death-hunter,10,6275,Action
...,...,...,...,...
6622221,76561198146468235,367090,1,Action
6622222,76561198146468235,367090,1,Indie
6622223,76561198146468235,367090,1,Sports
6622224,massimo23,448540,1,Adventure


In [16]:
# Sumo el tiempo jugado para cada género y jugador
# Ignoro la unidad de tiempo de los registros de playtime_forever. Dado que la respuesta que se pide es comparativa (medición relativa, no absoluta), el resultado no se verá afectado por la unidad elegida, asumiendo que todos los registros de la columna comparten la misma unidad de medida. 
df_grouped = df_join1.groupby(["user_id", "genres"])["playtime_forever"].sum()
df_grouped = df_grouped.to_frame().rename(columns={'playtime_forever': 'suma_play'})
df_new = df_grouped.reset_index()
df_new

,user_id,genres,suma_play
0,--000--,Action,139456
1,--000--,Adventure,11722
2,--000--,Casual,16135
3,--000--,Early Access,531
4,--000--,Free to Play,20435
...,...,...,...
620524,zzzmidmiss,RPG,181
620525,zzzmidmiss,Racing,9
620526,zzzmidmiss,Simulation,16
620527,zzzmidmiss,Sports,210


In [17]:
# Hallo los índices que corresponden al máximo de tiempo jugado de cada género
idx = df_new.groupby("genres")["suma_play"].idxmax()

In [18]:
# Me quedo con una lista de los géneros y usuarios correspondientes al género que tiene más tiempo jugado
result = df_new.loc[idx, ['user_id', 'genres', 'suma_play']]
result.reset_index(drop=True, inplace=True)
result

,user_id,genres,suma_play
0,Sp3ctre,Action,1686505
1,REBAS_AS_F-T,Adventure,2178420
2,Lickidactyl,Audio Production,109916
3,REBAS_AS_F-T,Casual,1220664
4,76561197978756659,Early Access,316969
5,76561198059330972,Education,65427
6,idonothack,Free to Play,808241
7,REBAS_AS_F-T,Indie,2397511
8,Evilutional,Massively Multiplayer,686969
9,Sky_Wolf,Photo Editing,21885


In [19]:
# El siguiente código filtra en el dataframe 'result' el año correspondiente al género que yo ingreso. Pruebo su funcionamiento ingresando un género cualquiera.

df_filtrado = result.loc[result["genres"] == genero, "user_id"]

In [20]:
usuario = df_filtrado.to_string(index=False)

In [21]:
# Ahora tengo que hallar el acumulado de horas jugadas por año para los usuarios del dataframe df_filtrado únicamente.
# Me quedo con la lista de usuarios de interés.
# En df_join2 tengo que filtrar los usuarios de interés. Los tomo de result. Previamente elimino a los repetidos.
result_unicos = result.drop_duplicates(subset="user_id")



 

In [22]:
print(result_unicos)

              user_id                 genres  suma_play
0             Sp3ctre                 Action    1686505
1        REBAS_AS_F-T              Adventure    2178420
2         Lickidactyl       Audio Production     109916
4   76561197978756659           Early Access     316969
5   76561198059330972              Education      65427
6          idonothack           Free to Play     808241
8         Evilutional  Massively Multiplayer     686969
9            Sky_Wolf          Photo Editing      21885
10        shinomegami                    RPG    1016710
11    DownSyndromeKid                 Racing     743445
12           jimmynoe             Simulation    1062130
14  76561197995725436                 Sports     193206
16  76561198073642113              Utilities     207651
17         ScottyG555       Video Production     168314
18            Xyphien         Web Publishing     142964


In [23]:
# Ahora me quedo solo con los user_id
result_unicos = result_unicos.drop(["genres", "suma_play"], axis=1)
result_unicos.reset_index(drop=True, inplace=True)


In [24]:
result_unicos

,user_id
0,Sp3ctre
1,REBAS_AS_F-T
2,Lickidactyl
3,76561197978756659
4,76561198059330972
5,idonothack
6,Evilutional
7,Sky_Wolf
8,shinomegami
9,DownSyndromeKid


In [25]:
# Filro df_join2 para quedarme solo con los usuarios que me interesan
df_prueba = df_join2[df_join2["user_id"].isin(result_unicos["user_id"])]

In [26]:
df_prueba

,genres,id,user_id,posted_year
9159,Action,71340,Lickidactyl,2015
13969,Action,730,Sp3ctre,2015
19704,Education,248170,76561198059330972,2015
19708,Utilities,248170,76561198059330972,2015
19712,Web Publishing,248170,76561198059330972,2015
20107,Action,222880,Xyphien,2014
20287,Indie,222880,Xyphien,2014
20467,Strategy,222880,Xyphien,2014
23017,Action,242760,Xyphien,2014
23184,Adventure,242760,Xyphien,2014


In [27]:
df_join3 = df_prueba.merge(result, on="genres", how="inner")

In [28]:
df_join3

,genres,id,user_id_x,posted_year,user_id_y,suma_play
0,Action,71340,Lickidactyl,2015,Sp3ctre,1686505
1,Action,730,Sp3ctre,2015,Sp3ctre,1686505
2,Action,222880,Xyphien,2014,Sp3ctre,1686505
3,Action,242760,Xyphien,2014,Sp3ctre,1686505
4,Action,252950,Sp3ctre,2015,Sp3ctre,1686505
5,Action,291550,Xyphien,2015,Sp3ctre,1686505
6,Action,271590,76561198059330972,2015,Sp3ctre,1686505
7,Action,354140,76561198059330972,2015,Sp3ctre,1686505
8,Action,209650,Sp3ctre,2015,Sp3ctre,1686505
9,Action,238090,Lickidactyl,2015,Sp3ctre,1686505


In [29]:
# Elimino las columnas user_id_x y suma_play
df_join3.drop(columns=['user_id_x', 'suma_play'])

,genres,id,posted_year,user_id_y
0,Action,71340,2015,Sp3ctre
1,Action,730,2015,Sp3ctre
2,Action,222880,2014,Sp3ctre
3,Action,242760,2014,Sp3ctre
4,Action,252950,2015,Sp3ctre
5,Action,291550,2015,Sp3ctre
6,Action,271590,2015,Sp3ctre
7,Action,354140,2015,Sp3ctre
8,Action,209650,2015,Sp3ctre
9,Action,238090,2015,Sp3ctre


In [30]:
# De df_ui1 me quedo con las columnas que voy a utilizar
df_ui2 = df_ui1.drop(columns=['user_id'])
df_ui2

,id,playtime_forever
0,10,6
1,30,7
2,300,4733
3,240,1853
4,3830,333
...,...,...
3246347,304930,677
3246348,227940,43
3246349,388490,3
3246350,521570,4


In [31]:
df_final = df_join3.merge(df_ui2, on='id', how='inner')
df_final1 = df_final.drop(columns=['user_id_x', 'suma_play'])

In [32]:
# Filtramos por usuario y género. Tomamos un usuario cualquiera, por ejemplo 'Sp3ctre', del dataframe previo
user_id_filtro = usuario
genres_filtro = genero
df_filtrado = df_final1.query("genres == @genres_filtro & user_id_y == @user_id_filtro")

In [33]:
# Sumamos los minutos de cada año (supongo que el tiempo está expresado en minutos)
df_agrupado = df_filtrado.groupby("posted_year")["playtime_forever"].sum()
# Lo transformamos en dataframe
df_nuevo = df_agrupado.reset_index()
# Dividimos por 60 para que 'playtime_forever' quede efectivamente expresado en horas
df_nuevo['playtime_forever'] = (df_nuevo['playtime_forever']/60).astype(int)

In [34]:
# Reemplazo nombre de columnas por los pedidos en el proyecto
df = df_nuevo.rename(columns={"posted_year": "Año", "playtime_forever": "Horas"})
# Para formatear la respuesta necesito transformar enteros en string
df = df.astype({"Año": str, "Horas": str})

In [35]:
# Convierto el dataframe en una lista de diccionarios
lista = df.to_dict(orient='records')
# Convierto la lista de diccionarios en una cadena de caracteres, para poder eliminar las comillas simples.
cadena = str(lista)
# Reemplazo las comillas simples por vacíos
cadena = cadena.replace("'", "")
# Corroboro que el formato coincide con el solicitado
print(f'{{"Usuario con más horas jugadas para Género {genero}" : {usuario}, "Horas jugadas":{cadena}}}')




{"Usuario con más horas jugadas para Género Action" : Sp3ctre, "Horas jugadas":[{Año: 2011, Horas: 23392}, {Año: 2012, Horas: 3373047}, {Año: 2014, Horas: 310658}, {Año: 2015, Horas: 14858388}]}
